## Propósito

En este ejercicio se trabaja con el tutorial [Neural Network Models for Combined Classification and Regression](https://machinelearningmastery.com/neural-network-models-for-combined-classification-and-regression/) desarrollado por el Dr. Brownlee.

Modelo combinado de regresión y clasificación multiclase utilizando `keras`

In [1]:
# regression mlp model for the abalone dataset
from pandas import read_csv
from numpy import unique
from numpy import argmax
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import plot_model

In [2]:
# load and summarize the abalone dataset
from matplotlib import pyplot
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/abalone.csv'
dataframe = read_csv(url, header=None)
# summarize shape
print(dataframe.shape)
# summarize first few lines
print(dataframe.head())

(4177, 9)
   0      1      2      3       4       5       6      7   8
0  M  0.455  0.365  0.095  0.5140  0.2245  0.1010  0.150  15
1  M  0.350  0.265  0.090  0.2255  0.0995  0.0485  0.070   7
2  F  0.530  0.420  0.135  0.6770  0.2565  0.1415  0.210   9
3  M  0.440  0.365  0.125  0.5160  0.2155  0.1140  0.155  10
4  I  0.330  0.255  0.080  0.2050  0.0895  0.0395  0.055   7


In [3]:
# split into input (X) and output (y) variables
X, y = dataframe.iloc[:, 1:-1], dataframe.iloc[:, -1]
X, y = X.astype('float'), y.astype('float')
n_features = X.shape[1]
print(n_features)

7


In [4]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [5]:
# define the keras model
model = Sequential()
model.add(Dense(20, input_dim=n_features, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(10, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='linear'))

In [6]:
# compile the keras model
model.compile(loss='mse', optimizer='adam')

In [7]:
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=0)

## Regression


In [8]:
# evaluate on test set
yhat = model.predict(X_test)
error = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % error)

MAE: 1.488


## Classification

In [9]:
# encode strings to integer
y = LabelEncoder().fit_transform(y)
n_class = len(unique(y))

In [10]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [11]:
# define the keras model
model = Sequential()
model.add(Dense(20, input_dim=n_features, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(10, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(n_class, activation='softmax'))

In [12]:
# compile the keras model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')


In [13]:
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=0)

In [35]:
# evaluate on test set
yhat = model.predict(X_test)
yhat = argmax(yhat, axis=-1).astype('int')
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

Accuracy: 0.270


## Combined Regression and Classification

In [36]:
# encode strings to integer
y_class = LabelEncoder().fit_transform(y)
n_class = len(unique(y_class))

In [37]:
# split data into train and test sets
X_train, X_test, y_train, y_test, y_train_class, y_test_class = train_test_split(X, y, y_class, 
                         test_size=0.33, random_state=1)


In [40]:
# input
visible = Input(shape=(n_features,))
hidden1 = Dense(20, activation='relu', kernel_initializer='he_normal')(visible)
hidden2 = Dense(10, activation='relu', kernel_initializer='he_normal')(hidden1)

In [41]:
# regression output
out_reg = Dense(1, activation='linear')(hidden2)

In [42]:
# classification output
out_clas = Dense(n_class, activation='softmax')(hidden2)

In [43]:
# define model
model = Model(inputs=visible, outputs=[out_reg, out_clas])

In [44]:
# compile the keras model
model.compile(loss=['mse','sparse_categorical_crossentropy'], optimizer='adam')

In [54]:
# plot graph of model
plot_model(model, to_file='model.png', show_shapes=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [46]:
# fit the keras model on the dataset
model.fit(X_train, [y_train,y_train_class], epochs=150, batch_size=32, verbose=0)

Epoch 1/150
88/88 - 1s - loss: 80.2515 - dense_14_loss: 77.0237 - dense_15_loss: 3.2278
Epoch 2/150
88/88 - 0s - loss: 39.9615 - dense_14_loss: 37.1110 - dense_15_loss: 2.8506
Epoch 3/150
88/88 - 0s - loss: 12.5511 - dense_14_loss: 9.9191 - dense_15_loss: 2.6320
Epoch 4/150
88/88 - 0s - loss: 10.6603 - dense_14_loss: 8.1334 - dense_15_loss: 2.5268
Epoch 5/150
88/88 - 0s - loss: 10.3949 - dense_14_loss: 7.8944 - dense_15_loss: 2.5006
Epoch 6/150
88/88 - 0s - loss: 10.1698 - dense_14_loss: 7.6787 - dense_15_loss: 2.4911
Epoch 7/150
88/88 - 0s - loss: 9.9728 - dense_14_loss: 7.4964 - dense_15_loss: 2.4765
Epoch 8/150
88/88 - 0s - loss: 9.7970 - dense_14_loss: 7.3540 - dense_15_loss: 2.4430
Epoch 9/150
88/88 - 0s - loss: 9.5996 - dense_14_loss: 7.2043 - dense_15_loss: 2.3954
Epoch 10/150
88/88 - 0s - loss: 9.4506 - dense_14_loss: 7.1064 - dense_15_loss: 2.3442
Epoch 11/150
88/88 - 0s - loss: 9.2890 - dense_14_loss: 6.9851 - dense_15_loss: 2.3039
Epoch 12/150
88/88 - 0s - loss: 9.1866 - den

In [47]:
# make predictions on test set
yhat1, yhat2 = model.predict(X_test)

### Evaluar regresión

In [48]:
# calculate error for regression model
error = mean_absolute_error(y_test, yhat1)
print('MAE: %.3f' % error)

MAE: 1.582


### Evaluar clasificación

In [50]:
# evaluate accuracy for classification model
yhat2 = argmax(yhat2, axis=-1).astype('int')
acc = accuracy_score(y_test_class, yhat2)
print('Accuracy: %.3f' % acc)

Accuracy: 0.273
